<a href="https://colab.research.google.com/github/D2J3D/PrLinAlg_Lab1/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import math
import random
import sympy

import numpy as np

In [22]:
def encrypt_matrix_former(key_word, alphabet):
    m = round(math.sqrt(len(key_word)))
    encrypt_matrix = [[0] * m for i in range(m)]
    j, k = 0, 0
    for i in range(len(key_word)):
        k %= m
        encrypt_matrix[j][k] = alphabet.index(key_word[i])
        if (i + 1) % m == 0:
            j += 1
        k += 1
    return np.array(encrypt_matrix)


In [23]:
def key_word_generator(alphabet, n):
    try:
        random_numbers = list(range(0, len(alphabet)))
        random.shuffle(random_numbers)
        key_word = ""
        for i in range(n**2):
            key_word += alphabet[random_numbers[i]]
        return key_word

    except ValueError:
        print(f"Не получится задать ключ длины n = {n}. Выбери меньшее значение n")


In [24]:
def mod_matrix(np_matrix, n):
    for i in range(np_matrix.shape[0]):
        for j in range(np_matrix.shape[1]):
            np_matrix[i][j] %= n
    return np_matrix


def mod_vector(np_matrix, n):
    for i in range(np_matrix.shape[0]):
        np_matrix[i] %= n
    return np_matrix


In [25]:
def matrix_to_text(np_matrix, alphabet):
    text = ""
    for i in range(np_matrix.shape[0]):
        for j in range(np_matrix.shape[1]):
            text += alphabet[np_matrix[i][j]]
    return text


def vector_to_text(np_matrix, alphabet):
    text = ""
    for i in range(np_matrix.shape[0]):
        text += alphabet[(np_matrix[i])]
    return text


In [26]:
def text_to_matrix(text, encrypt_matrix, alphabet):
    matrix_from_text = np.zeros((len(text)//encrypt_matrix.shape[1], encrypt_matrix.shape[1]), dtype=int)
    counter = 0
    for i in range(matrix_from_text.shape[0]):
        for j in range(matrix_from_text.shape[1]):
            matrix_from_text[i][j] = alphabet.index(text[counter])
            counter += 1
    return matrix_from_text


In [27]:
def encrypt_text(text_matrix, encrypt_matrix, alphabet):
    encrypted_text = ""
    for i in range(text_matrix.shape[0]):
        encrypt_string_matrix = mod_vector(np.dot(text_matrix[i], encrypt_matrix), len(alphabet))
        encrypted_text += vector_to_text(encrypt_string_matrix, alphabet)
    return encrypted_text


In [28]:
def decrypt_matrix_former(encrypt_matrix, n):
    return np.array(sympy.Matrix(encrypt_matrix).inv_mod(n))


def decrypt_text(encrypted_text, decrypt_matrix, alphabet):
    return encrypt_text(encrypted_text, decrypt_matrix, alphabet)


In [29]:
def spoil_message(text_matrix, alphabet):
    pos = [random.randint(1, len(alphabet)-1) for _ in range(3)]
    k = 0
    for i in range(text_matrix.shape[0]):
        for j in range(text_matrix.shape[1]):
            if k < 3:
                if pos[i] != text_matrix[i][j]:
                    text_matrix[i][j] = pos[k]
                else:
                    text_matrix[i][j] = (pos[k] - 1)
                k += 1
            else:
                return text_matrix
    return text_matrix


In [30]:
if __name__ == "__main__":
    # setup
    custom_alphabet = "а б в г д е ж з й к л м н о п р с т".split(" ")
    custom_alphabet.append(" ")
    n = len(custom_alphabet)
    custom_key_word = key_word_generator(custom_alphabet, 3) #3 - размерность матрицы-ключа
    custom_encrypt_matrix = encrypt_matrix_former(custom_key_word, custom_alphabet)
    message = text_to_matrix("ботай  матан", custom_encrypt_matrix, custom_alphabet)
    encrypted_text = encrypt_text(message, custom_encrypt_matrix, custom_alphabet)
    custom_decrypt_matrix = decrypt_matrix_former(custom_encrypt_matrix, len(custom_alphabet))
    final_decrypted_text = decrypt_text(text_to_matrix(encrypted_text, custom_encrypt_matrix, custom_alphabet),
                                        custom_decrypt_matrix, custom_alphabet)
    print(f"Матрица-ключ:\n{custom_encrypt_matrix}")
    print(f"Кодовое слово: {custom_key_word}")
    print(f"Сообщение, которое шифруем:{matrix_to_text(message, custom_alphabet)}")
    print(f"Зашифрованное сообщение:{encrypted_text}")
    #print(text_to_matrix("ботай  матан", custom_encrypt_matrix, custom_alphabet))
    #print(text_to_matrix(encrypted_text, custom_encrypt_matrix, custom_alphabet))
    print(f"Расшифрованное сообщение:{final_decrypted_text}")

    spoiled_message = spoil_message(text_to_matrix(encrypted_text, custom_encrypt_matrix, custom_alphabet), custom_alphabet)
    spoiled_message_text = matrix_to_text(spoiled_message, custom_alphabet)
    decrypted_spoiled_message = decrypt_text(spoiled_message, custom_decrypt_matrix, custom_alphabet)
    print(f"Испорченное сообщение:{spoiled_message_text}")
    print(f"Расшифрованное испорченное сообщение: {decrypted_spoiled_message}")


Матрица-ключ:
[[ 5 16 12]
 [ 8  1 17]
 [ 4 14  7]]
Кодовое слово: еснйбтдпз
Сообщение, которое шифруем:ботай  матан
Зашифрованное сообщение:жблгорзпдагг
Расшифрованное сообщение:ботай  матан
Испорченное сообщение:лйсгорзпдагг
Расшифрованное испорченное сообщение: страй  матан
